# Figment Fusion

`v1.0.0-alpha.1`

This is a pre-release version of Figment Fusion. You are welcome to share your feedback and contribute on [GitHub](https://github.com/rlaneth/figment-fusion).

---

Hello there! 👋

You've arrived at Figment Fusion, your way into the world of AI-based image generation!

This is a Colab notebook is intended for technology enthusiasts and is designed to provide insights on the process of running [Stable Diffusion](https://stability.ai/blog/stable-diffusion-announcement) with the [Diffusers library](https://github.com/huggingface/diffusers), allowing for exploration and customization while still remaining simple and fun to use.

Stable Diffusion is a state-of-the-art, free and open text-to-image model created by researchers from [CompVis](https://github.com/CompVis) and [Runway](https://runwayml.com/), with support from [Eleuther AI](https://www.eleuther.ai/), [LAION](https://laion.ai/) and [Stability AI](https://stability.ai/).

The Diffusers library by Hugging Face is used here with inspiration from the [Stable Diffusion with Diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb) demo.

## 🌱 Getting Started

Please execute all the steps in this notebook in the order they appear. Executing them out of order may cause errors or unexpected behavior.

Running the following code snippet is optional, but it helps you assert that you are connected to an appropriate runtime and allows you to discover which GPU type has been assigned for your current session.

In [ ]:
!nvidia-smi -L

The T4 and the P100 are two of the most frequently seen GPUs on Google Colab, the latter being faster and providing more VRAM than the former. It should be kept in mind that the [Google Colab FAQ](https://research.google.com/colaboratory/faq.html) states that the available GPU types may vary over time and, although access to resources is never guaranteed, users with an active subscription to the Pro and Pro+ tiers get priority access to the faster types.

## ⚙️ Settings

### Google Drive

The Google Drive integration, which is required for storing the Stable Diffusion model and, optionally, the generated images, may be configured via the following settings.

In [ ]:
#@title Base path

#@markdown The following parameter specifies which folder on Google Drive will be used to store all
#@markdown the data associated with Figment Fusion.

drive_base_path = '/Colab Data/Figment' #@param { "type": "string" }

In [ ]:
#@title Output

#@markdown The `drive_output_path` parameter determines at which folder within `drive_base_path` the
#@markdown generated images will be saved to. However, if `drive_output_autosave` is disabled, no
#@markdown output will be saved at all.

drive_output_path = '/output' #@param { "type": "string" }
drive_output_autosave = False #@param { "type": "boolean" }

In [ ]:
#@title Model

#@markdown These parameters determine at which folder within `drive_base_path` the Stable Diffusion
#@markdown model files will be stored. If you are running this notebook for the first time, you
#@markdown might need to enable `drive_model_download` so the files will be automatically retrieved
#@markdown from the [Hugging Face repository](https://huggingface.co/CompVis/stable-diffusion-v1-4).

drive_model_path = '/model' #@param { "type": "string" }
drive_model_download_missing = True #@param { "type": "boolean" }

drive_prefix_path = '/MyDrive'
drive_mount_path = '/content/drive'

drive_full_base_path = drive_mount_path + drive_prefix_path + drive_base_path
drive_full_model_path = drive_full_base_path + drive_model_path
drive_full_output_path = drive_full_base_path + drive_output_path

### Miscellaneous

In [ ]:
#@title Model

MODEL_MAP = {
  'Stable Diffusion v1.1': ['CompVis/stable-diffusion-v1-1', 'main'],
  'Stable Diffusion v1.2': ['CompVis/stable-diffusion-v1-2', 'main'],
  'Stable Diffusion v1.3': ['CompVis/stable-diffusion-v1-3', 'main'],
  'Stable Diffusion v1.4': ['CompVis/stable-diffusion-v1-4', 'main'],
  'Waifu Diffusion': ['hakurei/waifu-diffusion', 'main'],
  'FP16 - Stable Diffusion v1.1': ['CompVis/stable-diffusion-v1-1', 'fp16'],
  'FP16 - Stable Diffusion v1.2': ['CompVis/stable-diffusion-v1-2', 'fp16'],
  'FP16 - Stable Diffusion v1.3': ['CompVis/stable-diffusion-v1-3', 'fp16'],
  'FP16 - Stable Diffusion v1.4': ['CompVis/stable-diffusion-v1-4', 'fp16'],
  'FP16 - Waifu Diffusion': ['hakurei/waifu-diffusion', 'fp16']
}

#@markdown Here you can choose which model to be used for image generation. 

#@markdown Note that for retrieving each model, you are required to have an account on the
#@markdown [Hugging Face](https://huggingface.co/join) website and accept the terms on the
#@markdown corresponding repository, or else the download step will fail. For instance, if you wish
#@markdown to use Stable Diffusion v1.4 and is downloading it for the first time, you must accept
#@markdown the terms on [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4).

#@markdown The older versions of Stable Diffusion are made available here for experimental purposes.

#@markdown The half precision (`fp16`) variants of each model consume less VRAM, which is useful for
#@markdown running on less powerful GPUs such as the ones usually made available for users on the
#@markdown free tier of Colab.

#@markdown [Waifu Diffusion](https://huggingface.co/hakurei/waifu-diffusion) is a model based on
#@markdown Stable Diffusion v1.4 and fine-tuned for anime style art.

use_model = "FP16 - Stable Diffusion v1.4" #@param ["Stable Diffusion v1.1", "Stable Diffusion v1.2", "Stable Diffusion v1.3", "Stable Diffusion v1.4", "Waifu Diffusion", "FP16 - Stable Diffusion v1.1", "FP16 - Stable Diffusion v1.2", "FP16 - Stable Diffusion v1.3", "FP16 - Stable Diffusion v1.4", "FP16 - Waifu Diffusion"]

model_clone_repository = MODEL_MAP[use_model][0]
model_clone_branch = MODEL_MAP[use_model][1]

In [ ]:
#@title Safety filter

#@markdown By default, if generated content is determined to be unsafe (e.g. sexually explicit), it
#@markdown will not be shown or saved, being replaced with a black image instead. The following
#@markdown parameter allows you to chose whether to keep this feature enabled.

enable_safety_filter = True #@param { "type": "boolean" }

## 👮 Permissions

In [ ]:
#@title Google Drive { vertical-output: true }

#@markdown This step will require permission for accessing your files on Google
#@markdown Drive from within the Colab environment in order to store model files
#@markdown and, optionally, your generated images.

from google.colab import drive
from pathlib import Path

# Mount Google Drive folder in the Colab environment
drive.mount(drive_mount_path, force_remount=True)

# Create model and output path if they do not exist
Path(drive_full_model_path).mkdir(parents=True, exist_ok=True)
Path(drive_full_output_path).mkdir(parents=True, exist_ok=True)

In [ ]:
#@title Hugging Face { vertical-output: true }

#@markdown This step requires a Hugging Face access token which can be obtained on
#@markdown your [account settings page](https://huggingface.co/settings/tokens).
#@markdown The token is used for downloading the models, and is not required if you
#@markdown have disabled the `drive_model_download_missing` setting.

!pip install --quiet huggingface_hub

from google.colab import output
from huggingface_hub import notebook_login

output.enable_custom_widget_manager()
notebook_login()

## 📦 Requirements

In [ ]:
#@title Dependencies { vertical-output: true }

#@markdown This step downloads Python dependencies required by the Diffusers library.

!pip install --quiet diffusers==0.2.4 transformers scipy ftfy 'ipywidgets>=7,<8' python-slugify

In [ ]:
#@title File name generation

#@markdown This step defines a function which is used for generating names for
#@markdown the output files written to Google Drive.

import os
import binascii
from pathlib import Path
from slugify import slugify

def prompt_to_file_path(text_prompt):
  LOOP_MAX = 16
  slug = slugify(text_prompt)
  trun = slug[:16].rstrip('-')
  rand = binascii.b2a_hex(os.urandom(4)).decode()
  for _ in range(LOOP_MAX):
    name = '/{}-{}.png'.format(slug, rand)
    path = drive_full_output_path + name
    if not Path(path).exists():
      return path
  raise Exception('Unable to create an unique filename')

In [ ]:
#@title Create pipeline { vertical-output: true }

#@markdown This step creates and configures a Diffusers library
#@markdown `StableDiffusionPipeline`, which is required for the next steps.

#@markdown If the chosen generation model must be obtained from Hugging Face (e.g.
#@markdown because it is not already in your Google Drive), downloading will
#@markdown also be performed in this step.

#@markdown Please ensure that your Google Drive account has enough free space. For
#@markdown reference, the half precision version of the Stable Diffusion v1.4 model
#@markdown requires around 5.1 GB of storage.

import torch
from diffusers import StableDiffusionPipeline

def disabled_safety_filter(images, **kwargs):
  return images, False

if model_clone_branch == 'fp16':
  torch_dtype = torch.float16
else:
  torch_dtype = torch.float32

pipe = StableDiffusionPipeline.from_pretrained(
    model_clone_repository,
    torch_dtype=torch_dtype,
    cache_dir=drive_full_model_path,
    local_files_only=(not drive_model_download_missing),
    use_auth_token=drive_model_download_missing
  )

pipe = pipe.to('cuda')

if not enable_safety_filter:
  pipe.safety_checker = disabled_safety_filter

## 🎨 Usage

In [ ]:
#@title Text-to-Image { vertical-output: true }

text_prompt = '' #@param { "type": "string" }

height = 512 #@param {type:"slider", min:128, max:1024, step:128}
width = 512 #@param {type:"slider", min:128, max:1024, step:128}
num_inference_steps = 50 #@param {type:"slider", min:1, max:100, step:1}
guidance_scale = 7.5 #@param {type:"slider", min:0, max:30, step:0.5}
use_random_seed = True #@param { "type": "boolean" }
manual_seed = 0 #@param {type:"integer"}

generator = torch.Generator(device='cuda')
if use_random_seed:
  generator.seed()
else:
  generator.manual_seed(manual_seed)

with torch.autocast('cuda'):
  output = pipe(
    text_prompt,
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    generator=generator
  )['sample'][0]

# Save to Google Drive if the corresponding setting is enabled
if drive_output_autosave:
  image_path = prompt_to_file_path(text_prompt)
  output.save(image_path)

# Show output directly on Colab
output